In [8]:
import torch
torch.cuda.empty_cache()

In [9]:
import os
import copy
import torch
from tqdm import tqdm
import pandas as pd
import detectron2
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [10]:
# 기존 데이터셋 삭제
dataset_name = 'coco_trash_final'
if dataset_name in DatasetCatalog:
    DatasetCatalog.remove(dataset_name)
if dataset_name in MetadataCatalog:
    MetadataCatalog.remove(dataset_name)

# Register Dataset
try:
    register_coco_instances('coco_trash_final', {}, '/data/ephemeral/home/data/dataset/test.json', '/data/ephemeral/home/data/dataset/')
except AssertionError:
    pass

# try:
#     register_coco_instances('coco_trash_final', {}, '/data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/Split_data/valid_0_10.json', '/data/ephemeral/home/data/dataset/')
# except AssertionError:
#     pass

In [4]:
from detectron2.engine import (
    AMPTrainer,
    SimpleTrainer,
    default_argument_parser,
    default_setup,
    default_writers,
    hooks,
    launch,
)
from detectron2.modeling import GeneralizedRCNNWithTTA, GeneralizedRCNN
import logging
from detectron2.engine.defaults import create_ddp_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import inference_on_dataset, print_csv_format
from detectron2.config import LazyConfig, instantiate
from omegaconf import OmegaConf
import yaml
import pandas as pd
from tqdm import tqdm
import torch
import os

# 설정 파일 로드 (LazyConfig)
cfg = LazyConfig.load('/data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/vitdet_rcnn_cascade/output/config.yaml')
cfg.dataloader.test.dataset.names = 'coco_trash_final'

model = instantiate(cfg.model)
model.to(cfg.train.device)
model = create_ddp_model(model)
# 모델 가중치 로드
DetectionCheckpointer(model).load('/data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/vitdet_rcnn_cascade/output/model_0049999.pth')

# 테스트 데이터셋 로드
test_loader = instantiate(cfg.dataloader.test)

# 모델을 평가 모드로 설정
model.eval()

# 예측 수행
prediction_strings = []
file_names = []

for data in tqdm(test_loader):
    
    prediction_string = ''
    input=data[0]
    with torch.no_grad():
        outputs = model(data)[0]['instances']  # model에 올바른 형식으로 전달
    
    # 예측 결과 처리
    targets = outputs.pred_classes.cpu().tolist()
    boxes = [i.cpu().detach().numpy() for i in outputs.pred_boxes]
    scores = outputs.scores.cpu().tolist()
    
    for target, box, score in zip(targets, boxes, scores):
        prediction_string += (str(target) + ' ' + str(score) + ' ' + str(box[0]) + ' ' 
        + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' ')
    
    prediction_strings.append(prediction_string)
    file_names.append(input['file_name'].replace('/data/ephemeral/home/data/dataset/', ''))

# 제출 파일 생성
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.train.output_dir, 'submission_det2.csv'), index=False)


/opt/conda/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


[10/21 17:01:15 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/vitdet_rcnn_cascade/output/model_0049999.pth ...
[10/21 17:01:17 d2.data.datasets.coco]: Loaded 4871 images in COCO format from /data/ephemeral/home/data/dataset/test.json
[10/21 17:01:17 d2.data.build]: Distribution of instances among all 10 categories:
|   category    | #instances   |  category   | #instances   |  category  | #instances   |
|:-------------:|:-------------|:-----------:|:-------------|:----------:|:-------------|
| General trash | 0            |    Paper    | 0            | Paper pack | 0            |
|     Metal     | 0            |    Glass    | 0            |  Plastic   | 0            |
|   Styrofoam   | 0            | Plastic bag | 0            |  Battery   | 0            |
|   Clothing    | 0            |             |              |            |              |
|     total     | 0            |             |        

  0%|          | 0/4871 [00:04<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 768.00 MiB (GPU 0; 31.74 GiB total capacity; 2.11 GiB already allocated; 132.38 MiB free; 2.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [12]:
from detectron2.engine import (
    AMPTrainer,
    SimpleTrainer,
    default_argument_parser,
    default_setup,
    default_writers,
    hooks,
    launch,
)
from detectron2.modeling import GeneralizedRCNNWithTTA, GeneralizedRCNN
import logging
from detectron2.engine.defaults import create_ddp_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import inference_on_dataset, print_csv_format
from detectron2.config import LazyConfig, instantiate
from omegaconf import OmegaConf
from detectron2.config import get_cfg
import yaml
import pandas as pd
from tqdm import tqdm
import torch
import os

# 설정 파일 로드 (LazyConfig)
cfg = LazyConfig.load('/data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/vitdet_rcnn_cascade/output/config.yaml')
cfg.dataloader.test.dataset.names = 'coco_trash_final'

model = instantiate(cfg.model)
model.to(cfg.train.device)

# tta_cfg를 CfgNode로 생성
tta_cfg = get_cfg()
tta_cfg.TEST.AUG.ENABLED = True
tta_cfg.TEST.AUG.MIN_SIZES = (400, 500, 600, 700, 800)
tta_cfg.TEST.AUG.MAX_SIZE = 1333
tta_cfg.TEST.AUG.FLIP = True
tta_cfg.MODEL.KEYPOINT_ON = False
tta_cfg.MODEL.LOAD_PROPOSALS = False
tta_cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10  # 클래스 수에 맞게 조정
tta_cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5
tta_cfg.TEST.DETECTIONS_PER_IMAGE = 100

# 기존 cfg에서 필요한 설정을 복사
tta_cfg.MODEL.WEIGHTS = cfg.MODEL.WEIGHTS
tta_cfg.MODEL.DEVICE = cfg.MODEL.DEVICE

model = GeneralizedRCNNWithTTA(tta_cfg, model)

model = GeneralizedRCNNWithTTA(tta_cfg, model)
model = create_ddp_model(model)
# 모델 가중치 로드
DetectionCheckpointer(model).load('/data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/vitdet_rcnn_cascade/output/model_0049999.pth')

# 테스트 데이터셋 로드
test_loader = instantiate(cfg.dataloader.test)

# 모델을 평가 모드로 설정
model.eval()

# 예측 수행
prediction_strings = []
file_names = []

for data in tqdm(test_loader):
    
    prediction_string = ''
    input=data[0]
    with torch.no_grad():
        outputs = model(data)[0]['instances']  # model에 올바른 형식으로 전달
    
    # 예측 결과 처리
    targets = outputs.pred_classes.cpu().tolist()
    boxes = [i.cpu().detach().numpy() for i in outputs.pred_boxes]
    scores = outputs.scores.cpu().tolist()
    
    for target, box, score in zip(targets, boxes, scores):
        prediction_string += (str(target) + ' ' + str(score) + ' ' + str(box[0]) + ' ' 
        + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' ')
    
    prediction_strings.append(prediction_string)
    file_names.append(input['file_name'].replace('/data/ephemeral/home/data/dataset/', ''))

# 제출 파일 생성
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.train.output_dir, 'submission_det4.csv'), index=False)


[10/21 17:16:12 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/vitdet_rcnn_cascade/output/model_0049999.pth ...


Some model parameters or buffers are not found in the checkpoint:
model.backbone.net.blocks.0.attn.proj.{bias, weight}
model.backbone.net.blocks.0.attn.qkv.{bias, weight}
model.backbone.net.blocks.0.attn.{rel_pos_h, rel_pos_w}
model.backbone.net.blocks.0.mlp.fc1.{bias, weight}
model.backbone.net.blocks.0.mlp.fc2.{bias, weight}
model.backbone.net.blocks.0.norm1.{bias, weight}
model.backbone.net.blocks.0.norm2.{bias, weight}
model.backbone.net.blocks.1.attn.proj.{bias, weight}
model.backbone.net.blocks.1.attn.qkv.{bias, weight}
model.backbone.net.blocks.1.attn.{rel_pos_h, rel_pos_w}
model.backbone.net.blocks.1.mlp.fc1.{bias, weight}
model.backbone.net.blocks.1.mlp.fc2.{bias, weight}
model.backbone.net.blocks.1.norm1.{bias, weight}
model.backbone.net.blocks.1.norm2.{bias, weight}
model.backbone.net.blocks.10.attn.proj.{bias, weight}
model.backbone.net.blocks.10.attn.qkv.{bias, weight}
model.backbone.net.blocks.10.attn.{rel_pos_h, rel_pos_w}
model.backbone.net.blocks.10.mlp.fc1.{bias, weig

[10/21 17:16:14 d2.data.datasets.coco]: Loaded 4871 images in COCO format from /data/ephemeral/home/data/dataset/test.json
[10/21 17:16:14 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(1024, 1024), max_size=1024)]
[10/21 17:16:14 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[10/21 17:16:14 d2.data.common]: Serializing 4871 elements to byte tensors and concatenating them all ...
[10/21 17:16:14 d2.data.common]: Serialized dataset takes 0.61 MiB


  0%|          | 0/4871 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 132.00 MiB (GPU 0; 31.74 GiB total capacity; 2.00 GiB already allocated; 30.38 MiB free; 2.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF